# Σημειωματάριο πέμπτο: NLP Ετοιμάζοντας τα δεδομένα μας για την βαθύτερη ανάλυση

Σε αυτό το notebook ακολουθούμε όλα τα απαραίτητα βήματα προεπεξεργασίας των δεδομένων κειμένου (δηλαδή των 
tweets μας) πριν τα τροφοδοτήσουμε στην ανάλυσή μας.Δημιουργούμε ακόμη κάποιες νέες στήλες στο DataFrame που μπορεί να μας φανούν χρήσιμα στη συνέχεια (feature extraction). Είναι σημαντικό να σημειωθεί ότι τα βήματα αυτά πρέπει να εκτελεστούν στην σωστή σειρά ώστε να αποφύγουμε την απώλεια πληροφορίας ή την δημιουργία περισσότερου θορύβου στα δεδομένα μας. Για παράδειγμα ένα σημαντικό βήμα στην προεπεξεργασία κειμένου είναι η αφαίρεση των σημείων στίξης. Αυτό όμως πρέπει να γίνει αφού πρώτα διαχειριστούμε τα *#hashtags*, τα *em@ils* και τα *@mentions* γιατί αλλιώς από τη μία θα χάσουμε την πληροφορία που υπάρχει σε τέτοιου είδους μεταδεδομένα κι από την άλλη με την αφαίρεση των σημείων στίξης από αυτά, εκείνο που μένει ενδεχομένως να είναι θόρυβος.
 
Μερικά άρθρα και online υλικό που συμβουλευτήκαμε για την υλοποίηση των βημάτων προεπεξεργασίας είναι τα ακόλουθα: 
- https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79
- https://www.udemy.com/course/nlp-in-python/
- https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-text-pre-processing-with-python-example/
- https://machinelearningmastery.com/gentle-introduction-bag-words-model/

In [51]:
import pandas as pd
import nltk
import re
import string
import unicodedata
import unidecode

In [2]:
tweets = pd.read_csv('Data/tweets/tweets_2021_with_users_matched.csv', lineterminator='\n')

Θα δημιουργήσουμε μια νέα στήλη με όνομα *clean_text* όπου θα αποθηκεύονται τα tweets ύστερα από την εφαρμογή των βημάτων preprocessing. Αρχικά διατηρούμε και την στήλη των *tweets* όπου έχουμε τα πρωτότυπα κείμενα έτσι ώστε να μπορούμε να βλέπουμε τις διαφορές που προκύπτουν. Στο τέλος όμως, πριν από την ανάλυση, θα χρειαστεί να αφαιρεθούν.

Στην συνέχεια ακολουθούν ένα ένα τα βήματα του preprocessing.

In [3]:
# Lowercase
tweets["clean_text"] = tweets["text"].apply(lambda tweet: tweet.lower())

In [4]:
tweets = tweets.reindex(columns = ['id', 'text', 'clean_text', 'matched_CVE_IDs', 'created_at', 'retweet_count',
       'reply_count', 'like_count', 'quote_count', 'author_id', 'author_name',
       'author_username', 'author_description', 'author_followers_count',
       'author_following_count', 'author_tweet_count', 'author_listed_count'])

In [48]:
# Δημιουργία νέας στήλης όπου αποθηκεύουμε το μέγεθος του tweet σε χαρακτήρες

tweets['size_of_tweet_in_characters'] = tweets['text'].apply(lambda tweet: len(tweet))

In [5]:
# Διαχείριση υπερσυνδέσμων.
# https://stackoverflow.com/a/54086404

# Πρώτα τους παίρνουμε από το text και τους κρατάμε σε μια νέα στήλη
url_pattern = '''(?:(?:https?|ftp):\/\/|\b(?:[a-z\d]+\.))(?:(?:[^\s()<>]+|\((?:[^\s()<>]+|(?:\([^\s()<>]+\)))?\))+(?:\((?:[^\s()<>]+|(?:\(?:[^\s()<>]+\)))?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))?'''
tweets['number_of_links_included'] = tweets['text'].apply(lambda tweet: len(re.findall(url_pattern, tweet)))
tweets['links_included'] = tweets['text'].apply(lambda tweet: re.findall(url_pattern, tweet))


In [6]:
# Και μετά τους διαγράφουμε από το clean_text
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: re.sub(url_pattern, '', tweet))

In [7]:
# Διαχείριση emails

# Πρώτα τα παίρνουμε από το text και τους κρατάμε σε μια νέα στήλη
email_pattern = '([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z+_-]+)'
tweets['number_of_emails_included'] = tweets['text'].apply(lambda tweet: len(re.findall(email_pattern, tweet)))
tweets['emails_included'] = tweets['text'].apply(lambda tweet: re.findall(email_pattern, tweet))


In [8]:
len(tweets[tweets['number_of_emails_included']>0])

24

In [9]:
# Και μετά τα διαγράφουμε από το clean_text
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: re.sub(email_pattern, '', tweet))

In [10]:
# Αφαίρεση αχρείαστων κενών (μπορεί να έχουν δημιουργηθεί ξανά)
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join(tweet.split()))

In [11]:
tweets.head(1)

,id,text,clean_text,matched_CVE_IDs,created_at,retweet_count,reply_count,like_count,quote_count,author_id,...,author_username,author_description,author_followers_count,author_following_count,author_tweet_count,author_listed_count,number_of_links_included,links_included,number_of_emails_included,emails_included
0,1458428897368940553,CVE-2021-34582\n\nIn Phoenix Contact FL MGUARD...,cve-2021-34582 in phoenix contact fl mguard 11...,['CVE-2021-34582'],2021-11-10T13:38:40.000Z,0,0,0,0,941389496771399680,...,VulmonFeeds,Vulnerability Feed Bot (tweets new and some ol...,1830,2,85176,46,1,[https://t.co/YDYnY307QV],0,[]


In [12]:
# Διαχείριση mentions
# Το εφαρμόζουμε στο clean_text γιατί από εκεί έχουμε αφαιρέσει τα emails οπότε δεν υπάρχει κίμδυνος να πάρουμε 
# και email domains.
# Πρώτα τα παίρνουμε και τα κρατάμε σε μια νέα στήλη
# Χρησιμοποιούμε το replace για να αφαιρέσουμε τπν χαρακτήρα ':' που υπάρχει σε αρκετά mentions. Δεν μπορεί άλλωστε 
# να είναι τμήμα του username καθώς διαβάζουμε ότι "A username can only contain alphanumeric 
# characters (letters A-Z, numbers 0-9) with the exception of underscores"

tweets['number_of_mentions_included'] = tweets['clean_text'].apply(lambda tweet: len([word for word in tweet.split() if word.startswith('@')]))
tweets['mentions_included'] = tweets['clean_text'].apply(lambda tweet: [word.replace(':','') for word in tweet.split() if word.startswith('@')])

In [13]:
# Και μετά τα διαγράφουμε από το clean_text

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join([word for word in tweet.split() if not word.startswith('@')]))

In [14]:
tweets[tweets['number_of_mentions_included']>0]

,id,text,clean_text,matched_CVE_IDs,created_at,retweet_count,reply_count,like_count,quote_count,author_id,...,author_followers_count,author_following_count,author_tweet_count,author_listed_count,number_of_links_included,links_included,number_of_emails_included,emails_included,number_of_mentions_included,mentions_included
98,1458406769668067329,RT @JohnAlvahCoe: ZOHO ManageEngine ADSelfServ...,rt zoho manageengine adselfservice plus versio...,['CVE-2021-40539'],2021-11-10T12:10:44.000Z,0,0,0,0,234435858,...,2561,2005,37649,101,1,[https://t.co/XCyrSUrjvW],0,[],4,"[@johnalvahcoe, @msftsecintel, @paloaltontwks,..."
105,1458402615172161539,ZOHO ManageEngine ADSelfService Plus versions ...,zoho manageengine adselfservice plus versions ...,['CVE-2021-40539'],2021-11-10T11:54:13.000Z,0,0,1,0,64812320,...,363,610,8235,95,1,[https://t.co/8E87aM1IoI],0,[],3,"[@msftsecintel, @paloaltontwks, @blacklotuslabs]"
328,1458343878176788489,#SonicWall RT @SonicWallAlerts: SonicAlert: Th...,#sonicwall rt sonicalert: the sonicwall captur...,['CVE-2021-35215'],2021-11-10T08:00:49.000Z,0,0,0,0,123521998,...,573,1051,41911,5,1,[https://t.co/LwtiUeAvoF],0,[],1,[@sonicwallalerts]
333,1458342506970615809,For my Danish followers: @DCIS_SUND: I bør hav...,for my danish followers: i bør have et vågent ...,['CVE-2021-31886'],2021-11-10T07:55:23.000Z,2,1,4,0,18669683,...,12890,829,11936,504,1,[https://t.co/Aj3ItkOQlV],0,[],1,[@dcis_sund]
351,1458342250245722117,Our latest @McAfee_ATR analysis looks at CVE-2...,our latest analysis looks at cve-2021-38666 ye...,['CVE-2021-38666'],2021-11-10T07:54:21.000Z,6,0,8,0,100983645,...,13758,629,9753,461,1,[https://t.co/G7eZCc0gVD],0,[],3,"[@mcafee_atr, @w3knight, @eayep]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199079,1347658302424023041,#Vulnerabilities: #LaminasProject Laminas-Http...,#vulnerabilities: #laminasproject laminas-http...,['CVE-2021-3007'],2021-01-08T21:35:52.000Z,2,0,0,0,172545314,...,937,1,19423,591,2,"[https://t.co/IP3Zl1LcUo, https://t.co/lv2ayR3...",0,[],1,[@github]
199224,1347473290902835200,ZF3/Laminas-http (CVE-2021-3007) under active ...,zf3/laminas-http (cve-2021-3007) under active ...,['CVE-2021-3007'],2021-01-08T09:20:42.000Z,0,0,2,0,2367201715,...,80,259,246,3,0,[],0,[],1,[@spazef0rze]
199381,1347131121457377281,✅ Google publica actualización de seguridad pa...,✅ google publica actualización de seguridad pa...,['CVE-2021-0316'],2021-01-07T10:41:02.000Z,0,0,0,0,390945473,...,355,190,5010,3,1,[https://t.co/BanXLw1teL],0,[],1,[@securityaffairs]
199534,1346390178865156097,"An untrusted deserialization vulnerability, tr...","an untrusted deserialization vulnerability, tr...",['CVE-2021-3007'],2021-01-05T09:36:48.000Z,0,0,0,0,3228055976,...,717,2055,2320,38,1,[https://t.co/wMx9oeVkM5],0,[],2,"[@adminahead, @katzionglobal]"


In [15]:
# Διαγραφή των 'rt'

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join([word for word in tweet.split() if not word == 'rt']))

In [16]:
# Διαχείριση hashtags
# Πρώτα τα κρατάμε σε μια νέα στήλη 

tweets['number_of_hashtags_included'] = tweets['clean_text'].apply(lambda tweet: len([word for word in tweet.split() if word.startswith('#')]))
tweets['hashtags_included'] = tweets['clean_text'].apply(lambda tweet: [word for word in tweet.split() if word.startswith('#')])

In [17]:
# Και μετά τα διαγράφουμε από το clean_text

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join([word for word in tweet.split() if not word.startswith('#')]))

In [18]:
len(tweets[tweets['number_of_hashtags_included']>0])

34206

In [55]:
tweets.iloc[75555].text

['new',
 'cve-2021-24014',
 'multipl',
 'instanc',
 'improp',
 'neutral',
 'input',
 'web',
 'page',
 'gener',
 'vulner',
 'fortisandbox',
 '400',
 'may',
 'allow',
 'unauthent',
 'attack',
 'perform',
 'xss',
 'attack',
 'click',
 'sever',
 'medium']

In [20]:
# https://datagy.io/python-remove-punctuation-from-string/
# Είναι σημαντικό να αφαιρέσουμε τα σημεία στίξης αφού πρώτα διαχειριστούμε υπερσυνδέσμους, hashtags, emojis
# και mentions.
def remove_punctuations(text):
    '''
    Με την συνάρτηση αυτή αφαιρούμε τα σημεία στίξης.
    Η παύλα δεν αφαιρείται για να παραμείνει στο CVE αν και θα ήταν καλό να μπορούμε να αφαιρέσουμε όλες τις παύλες
    που δεν είναι μέρος του CVE.
    '''
    my_punctuations = string.punctuation
    new_text = text.translate(str.maketrans('', '', string.punctuation.replace('-','')))
    return new_text

In [21]:
# Αφαίρεση σημείων στίξης
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: remove_punctuations(tweet))

In [22]:
# κειμενο απο αλλες γλωσσες
# τονισμενες λεξεις
# emojis

In [23]:
def remove_accented_chars(text):
    """Συνάρτηση για αφαίρεση των τόνων, π.χ. από café θα γίνει μετατροπή σε cafe.
       Αυτό είναι απαραίτητο ώστε οι αλγόριθμοι να μην αντιμετωπίσουν ως ξεχωριστές λέξεις δύο ίδιες λόγω διαφορετικού 
       τονισμού.
       SOS: αυτό αφαιρεί και emoji και κείμενο από άλλες γλωσσες και επίσης και άλλους χαρακτήρες όπως # και @ 
       οπότε αφαιρείται."""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [24]:
# Παράδειγμα:
x = 'Áccěntěd těxt rks Browser に複数の脆弱性 🚨 NEW: καλημέρα CVE-2021-24014 🚨 Multiple #git @github' 
x = remove_accented_chars(x)
print(x)
' '.join(x.split())

Accented text rks Browser   NEW:  CVE-2021-24014  Multiple #git @github


'Accented text rks Browser NEW: CVE-2021-24014 Multiple #git @github'

In [25]:
# υπάρχει και αυτή η επιλογή για το transiteration
unidecode.unidecode("Καλημέρα!")

'Kalemera!'

In [26]:
# Αφαίρεση τονισμένων, emojis, και κειμένου από άλλες γλώσσες 
tweets["clean_text"] = tweets["clean_text"].apply(lambda tweet: remove_accented_chars(tweet))

In [27]:
# Αφαίρεση αχρείαστων κενών (μπορεί να έχουν δημιουργηθεί από την αφαίρεση emojis...)
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join(tweet.split()))

In [34]:
# Tokenization 

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: nltk.tokenize.word_tokenize(tweet))

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/terzisilias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
# Αφαίρεση stopwords

stop_words = set(nltk.corpus.stopwords.words('english'))
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: [word for word in tweet if word not in stop_words])

In [42]:
# Stemming

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: [nltk.stem.PorterStemmer().stem(word) for word in tweet])

In [ ]:
#pipeline για το model

In [32]:
# Απομένουν:

# διαχείριση αριθμών (εκτός από αυτών που είναι σε CVE)
# καλύτερα θα ήταν να μεταφράζουμε το κείμενο άλλων γλωσσών που δεν είναι σε λατινικούς χαρακτήρες αντί να το διαγράφουμε; 
# καλύτερα θα ήταν να "μεταφράζουμε" τα emojis;
# χρειάζονται τα Common words removal  και Rare words removal  πρίν το μοντέλο; (από ινδό)
# Spelling Correction ? (από το notebook του ινδού)
# κάνε και ένα wordcloud τώρα που είναι φτιαγμένα τα data